In [1]:
def replace_symbol(text):
    temp_text = text.replace("_"," ")
    temp_text = temp_text.replace("-LRB-","")
    temp_text = temp_text.replace("-RRB-","")
    temp_text = temp_text.replace("-LSB-","")
    temp_text = temp_text.replace("-RSB-","")
    temp_text = temp_text.replace("\\","")
    temp_text = temp_text.replace("/","")
    temp_text = temp_text.replace(")","")   
    temp_text = temp_text.replace("(","") 
    return temp_text

In [2]:
# preprocessing
from collections import defaultdict
from collections import Counter
import time

wiki = []
topics = {}
replaced_topics = []
topic_list = []

error_sentence = 0

time_start_out = time.time()
document_num = 109
for k in range(document_num):
    #number = str(i+1)
    #filename = 'wiki-pages-text/wiki-%s.txt' % (number.zfill(3))
    filename = 'wiki-pages-text/wiki-%03d.txt' % (k+1)    
    with open(filename, 'r', encoding='UTF-8') as f:
        content_lines = f.readlines() 
        time_start = time.time()
        print('processing %d' % (k+1))
        for i in range(len(content_lines)):
        #for i in range(10):
            wiki_sentence = content_lines[i].split(" ",2)
            topic = wiki_sentence[0]
            
            if (not wiki_sentence[1].isdigit()):
                error_sentence +=1
                continue
                
            sentence_number = int(wiki_sentence[1])
            if topic not in topics:
                topic_id = len(topics)
                topics[topic] = topic_id
                
                topic_list.append(topic)
                                
                replaced_topic = replace_symbol(topic)
                replaced_topics.append(replaced_topic)
                
                content = replace_symbol(wiki_sentence[2])
                wiki.append({sentence_number:content})
            else :
                topic_id = topics[topic]
                
                content = replace_symbol(wiki_sentence[2])
                
                wiki[topic_id][sentence_number] = content
                
        time_end = time.time()
        cost_time = time_end - time_start
        print('time cost',cost_time,'s')
time_end_out = time.time()
avg_cosumed_time = (time_end_out-time_start_out)/document_num
print('average time', avg_cosumed_time, 's')
print('error_sentence_count',error_sentence)

processing 1
time cost 0.8464090824127197 s
processing 2
time cost 1.0276422500610352 s
processing 3
time cost 0.943763017654419 s
processing 4
time cost 1.0095601081848145 s
processing 5
time cost 1.1043956279754639 s
processing 6
time cost 1.118170976638794 s
processing 7
time cost 1.006964921951294 s
processing 8
time cost 1.0531728267669678 s
processing 9
time cost 1.0450551509857178 s
processing 10
time cost 1.1345329284667969 s
processing 11
time cost 0.9945368766784668 s
processing 12
time cost 1.021421194076538 s
processing 13
time cost 1.0205998420715332 s
processing 14
time cost 1.0897247791290283 s
processing 15
time cost 1.120332956314087 s
processing 16
time cost 1.0315730571746826 s
processing 17
time cost 1.0268638134002686 s
processing 18
time cost 1.0589141845703125 s
processing 19
time cost 1.0341389179229736 s
processing 20
time cost 1.0521752834320068 s
processing 21
time cost 1.0584170818328857 s
processing 22
time cost 1.0937409400939941 s
processing 23
time cost 

In [3]:
## block for generate inverted index

## if inverted index is already generated, you need to delete the index2/ folder.

import os
import lucene
from java.io import File
from org.apache.lucene.analysis.standard import StandardAnalyzer
from org.apache.lucene.document import Document, Field, FieldType
from org.apache.lucene.index import IndexWriter, IndexWriterConfig, IndexOptions, IndexReader, DirectoryReader
from org.apache.lucene.store import SimpleFSDirectory, FSDirectory
from org.apache.lucene.util import Version
from org.apache.lucene.search import IndexSearcher
from org.apache.lucene.queryparser.classic import QueryParser

lucene.initVM()
INDEX_DIR = 'index2/'
store = SimpleFSDirectory(File(INDEX_DIR).toPath())
index_writer = IndexWriter(store, IndexWriterConfig(StandardAnalyzer()))

t1 = FieldType()
t1.setOmitNorms(False)
t1.setStored(True)
t1.setTokenized(True)
t1.setIndexOptions(IndexOptions.DOCS_AND_FREQS)

time_start_generate = time.time()

for n, content in enumerate(replaced_topics):  # read in doc here
    doc = Document()
    doc.add(Field("topic_id", str(n), t1))
    doc.add(Field("topic_content", content, t1))    

    index_writer.addDocument(doc)   # Add the document to the index

print("%r docs in index" % index_writer.numDocs())
index_writer.commit()
index_writer.close()
time_end_generate = time.time()
print("take ", time_end_generate - time_start_generate, " s to generate inverted index")



5396106 docs in index
take  167.7132842540741  s to generate inverted index


In [6]:
#generate trainnig set for training

searcher = IndexSearcher(DirectoryReader.open(store))

import json
import jsonlines
import unicodedata
import time
import spacy
nlp = spacy.load('en_core_web_lg')

def data_to_jsonl(data,file):
    with jsonlines.open(file, mode='w') as writer:
        for values in data:
            writer.write({"sentence1": values[1], "sentence2": values[0], "gold_label": values[2]})

def generate_training(input_file,output_file):
    training_set = json.load(open(input_file))
    print("input_file:",input_file)
    data = []
    process_counter = 0
    time_start = time.time()
    
    for key,value in training_set.items():
        process_counter += 1
        if (process_counter % 100 == 0):
            print("have process:",process_counter,"speed:(instance/s)",process_counter/(time.time()-time_start))
            
        claim = value['claim']
        claim = replace_symbol(claim)

        label = value['label']
        evidences = value['evidence']

        if len(evidences) != 0:
            for evidence in evidences:
                sentence_num = int(evidence[1])
                sentence_topic = unicodedata.normalize('NFC',evidence[0])
                topic_id = topics[sentence_topic]
                sentence_content = wiki[topic_id][sentence_num]
                data.append((claim,sentence_content,label))
        else:
            doc_claim = nlp(claim)
            query = ""
            for token in doc_claim:
                if token.text[0].isupper() and token.pos_ != "DET":
                    query += (token.text + " ")
                elif token.pos_ in {"NOUN",'PROPN','NUM'}:
                    if token.pos_ != 'NUM' or token.text.isalnum():
                        query += (token.text + " ")   

            if(len(query) == 0):
                print("this claim has no query:", claim)
                query = claim

            my_query = QueryParser("topic_content", StandardAnalyzer()).parse(query)                                                            # type query here
            MAX = 1
            total_hits = searcher.search(my_query, MAX).scoreDocs
            if len(total_hits) == 0:
                print("total_hits:",len(total_hits),"query:",query,"claim:",claim)
                continue        
            topic_id = int(searcher.doc(total_hits[0].doc).get("topic_id"))
            sentence_counter = 0
            for num, content in wiki[topic_id].items():
                sentence_counter += 1
                if sentence_counter > 5:
                    break
                data.append((claim,content,label))
    data_to_jsonl(data,output_file)
    
generate_training("train.json","train_nlp.jsonl")
generate_training("devset.json","devset_nlp.jsonl")


input_file: train.json
have process: 100 speed:(instance/s) 85.87795035615063
have process: 200 speed:(instance/s) 85.22300740638696
have process: 300 speed:(instance/s) 90.3684693637399
have process: 400 speed:(instance/s) 97.51533520355926
have process: 500 speed:(instance/s) 100.62473981969276
have process: 600 speed:(instance/s) 102.31502890055907
have process: 700 speed:(instance/s) 104.92523180655874
have process: 800 speed:(instance/s) 105.51056532231783
have process: 900 speed:(instance/s) 107.3450868243243
have process: 1000 speed:(instance/s) 109.46045595718188
have process: 1100 speed:(instance/s) 110.50950040003916
have process: 1200 speed:(instance/s) 110.56096573126469
have process: 1300 speed:(instance/s) 110.14388590500626
have process: 1400 speed:(instance/s) 112.38270154511379
have process: 1500 speed:(instance/s) 112.32556520248737
have process: 1600 speed:(instance/s) 112.78629312926068
have process: 1700 speed:(instance/s) 114.9249421157812
have process: 1800 speed

have process: 14500 speed:(instance/s) 151.1471343623076
have process: 14600 speed:(instance/s) 151.30904674030995
have process: 14700 speed:(instance/s) 151.55061776308307
have process: 14800 speed:(instance/s) 151.67064784499962
have process: 14900 speed:(instance/s) 151.79641128426945
have process: 15000 speed:(instance/s) 152.10001902236604
have process: 15100 speed:(instance/s) 152.2483250896809
have process: 15200 speed:(instance/s) 152.54217721608347
have process: 15300 speed:(instance/s) 152.5981248190638
have process: 15400 speed:(instance/s) 152.8791569756681
have process: 15500 speed:(instance/s) 152.75455677388317
have process: 15600 speed:(instance/s) 152.99126583132664
have process: 15700 speed:(instance/s) 153.33758090649692
have process: 15800 speed:(instance/s) 153.41839586209932
have process: 15900 speed:(instance/s) 153.48319537714355
have process: 16000 speed:(instance/s) 153.47981953550604
have process: 16100 speed:(instance/s) 153.52399228880412
have process: 1620

have process: 28800 speed:(instance/s) 165.90611302063812
have process: 28900 speed:(instance/s) 165.8647298402123
have process: 29000 speed:(instance/s) 166.0078414833892
have process: 29100 speed:(instance/s) 166.12847391800133
have process: 29200 speed:(instance/s) 166.10327128503405
have process: 29300 speed:(instance/s) 166.23185112868796
have process: 29400 speed:(instance/s) 166.4117698189458
have process: 29500 speed:(instance/s) 166.4633300482787
have process: 29600 speed:(instance/s) 166.6388970513215
have process: 29700 speed:(instance/s) 166.61220248042818
have process: 29800 speed:(instance/s) 166.51573152878953
have process: 29900 speed:(instance/s) 166.68655754424842
have process: 30000 speed:(instance/s) 166.83431004635005
have process: 30100 speed:(instance/s) 166.92966409970455
have process: 30200 speed:(instance/s) 167.05551309033817
have process: 30300 speed:(instance/s) 167.23384782126485
have process: 30400 speed:(instance/s) 167.3080386949739
have process: 30500 

have process: 43000 speed:(instance/s) 176.3754168166378
have process: 43100 speed:(instance/s) 176.44486581673368
have process: 43200 speed:(instance/s) 176.52608065765983
have process: 43300 speed:(instance/s) 176.55356421472183
have process: 43400 speed:(instance/s) 176.56248473088456
have process: 43500 speed:(instance/s) 176.49561045724974
have process: 43600 speed:(instance/s) 176.5480736813336
have process: 43700 speed:(instance/s) 176.58112852472942
have process: 43800 speed:(instance/s) 176.66086514307426
have process: 43900 speed:(instance/s) 176.8405184207191
have process: 44000 speed:(instance/s) 176.88633858682775
have process: 44100 speed:(instance/s) 176.83176286062275
have process: 44200 speed:(instance/s) 176.94236001846434
have process: 44300 speed:(instance/s) 176.86564038619983
have process: 44400 speed:(instance/s) 176.96705169752732
have process: 44500 speed:(instance/s) 177.09200389082497
have process: 44600 speed:(instance/s) 177.11312579232722
have process: 447

have process: 57200 speed:(instance/s) 182.7898323560715
have process: 57300 speed:(instance/s) 182.77623955953416
have process: 57400 speed:(instance/s) 182.78073210165016
have process: 57500 speed:(instance/s) 182.82636581766104
have process: 57600 speed:(instance/s) 182.8185791258513
have process: 57700 speed:(instance/s) 182.86592014815642
have process: 57800 speed:(instance/s) 182.88185810406296
have process: 57900 speed:(instance/s) 182.89452043900224
have process: 58000 speed:(instance/s) 182.97255971857464
have process: 58100 speed:(instance/s) 183.053573103964
have process: 58200 speed:(instance/s) 183.09882051971022
have process: 58300 speed:(instance/s) 183.2180516879919
have process: 58400 speed:(instance/s) 183.21986970321248
have process: 58500 speed:(instance/s) 183.1975009380082
have process: 58600 speed:(instance/s) 183.33823778194417
have process: 58700 speed:(instance/s) 183.30558234576196
have process: 58800 speed:(instance/s) 183.32199694483998
have process: 58900 

...

have process: 85700 speed:(instance/s) 191.26967186262974
have process: 85800 speed:(instance/s) 191.23813798225171
have process: 85900 speed:(instance/s) 191.24878332474984
have process: 86000 speed:(instance/s) 191.2597033434392
have process: 86100 speed:(instance/s) 191.23337982160336
have process: 86200 speed:(instance/s) 191.19979340782982
have process: 86300 speed:(instance/s) 191.1778064843082
have process: 86400 speed:(instance/s) 191.190319213657
have process: 86500 speed:(instance/s) 191.2206691404635
have process: 86600 speed:(instance/s) 191.20698504915038
have process: 86700 speed:(instance/s) 191.2123403847979
have process: 86800 speed:(instance/s) 191.23452787969418
have process: 86900 speed:(instance/s) 191.23500513154454
have process: 87000 speed:(instance/s) 191.23318245196447
have process: 87100 speed:(instance/s) 191.2490855856208
have process: 87200 speed:(instance/s) 191.1847043140559
have process: 87300 speed:(instance/s) 191.1254259471975
have process: 87400 spe

have process: 100000 speed:(instance/s) 193.5222176046839
have process: 100100 speed:(instance/s) 193.5514949086557
have process: 100200 speed:(instance/s) 193.55337372367555
have process: 100300 speed:(instance/s) 193.58345837330248
have process: 100400 speed:(instance/s) 193.52986885399199
have process: 100500 speed:(instance/s) 193.57596112445304
have process: 100600 speed:(instance/s) 193.62285793109166
have process: 100700 speed:(instance/s) 193.65551301834486
have process: 100800 speed:(instance/s) 193.66942119765807
have process: 100900 speed:(instance/s) 193.73080684415294
have process: 101000 speed:(instance/s) 193.73591514045523
have process: 101100 speed:(instance/s) 193.77301313128837
have process: 101200 speed:(instance/s) 193.78151918095512
have process: 101300 speed:(instance/s) 193.79583580666863
have process: 101400 speed:(instance/s) 193.81124557933276
have process: 101500 speed:(instance/s) 193.7879649575131
have process: 101600 speed:(instance/s) 193.7698511074849
h

have process: 113900 speed:(instance/s) 195.55828681225137
have process: 114000 speed:(instance/s) 195.55547988098044
have process: 114100 speed:(instance/s) 195.5572849867899
have process: 114200 speed:(instance/s) 195.61149011886872
have process: 114300 speed:(instance/s) 195.64634366073946
have process: 114400 speed:(instance/s) 195.66911315162315
have process: 114500 speed:(instance/s) 195.66584234807263
have process: 114600 speed:(instance/s) 195.55629109948248
have process: 114700 speed:(instance/s) 195.6286943963561
have process: 114800 speed:(instance/s) 195.6642516499048
have process: 114900 speed:(instance/s) 195.6971190094058
have process: 115000 speed:(instance/s) 195.746778245555
have process: 115100 speed:(instance/s) 195.76717416687683
have process: 115200 speed:(instance/s) 195.77280323730633
have process: 115300 speed:(instance/s) 195.7830939401053
have process: 115400 speed:(instance/s) 195.81139464709446
have process: 115500 speed:(instance/s) 195.77267502087528
have

have process: 127900 speed:(instance/s) 197.08095335793533
have process: 128000 speed:(instance/s) 197.08822200524693
have process: 128100 speed:(instance/s) 197.02012785407499
have process: 128200 speed:(instance/s) 196.9969087002389
have process: 128300 speed:(instance/s) 196.98587106434107
have process: 128400 speed:(instance/s) 196.98467695854956
have process: 128500 speed:(instance/s) 196.97546426729465
have process: 128600 speed:(instance/s) 196.9427938720338
have process: 128700 speed:(instance/s) 196.95428934132968
have process: 128800 speed:(instance/s) 196.9782236922268
have process: 128900 speed:(instance/s) 197.0177024629793
have process: 129000 speed:(instance/s) 196.9962651071884
have process: 129100 speed:(instance/s) 197.00472544276082
have process: 129200 speed:(instance/s) 196.97867275502384
have process: 129300 speed:(instance/s) 197.01172726773342
have process: 129400 speed:(instance/s) 196.99675910289506
have process: 129500 speed:(instance/s) 197.00934430803542
ha

have process: 141900 speed:(instance/s) 198.0842818051851
have process: 142000 speed:(instance/s) 198.0746615289834
have process: 142100 speed:(instance/s) 198.08799224763058
have process: 142200 speed:(instance/s) 198.08330797082436
have process: 142300 speed:(instance/s) 198.12627202281527
have process: 142400 speed:(instance/s) 198.11656201353577
have process: 142500 speed:(instance/s) 198.0928967299174
have process: 142600 speed:(instance/s) 198.07624760804504
have process: 142700 speed:(instance/s) 198.10429708309536
have process: 142800 speed:(instance/s) 198.14532832542793
have process: 142900 speed:(instance/s) 198.19781288735604
have process: 143000 speed:(instance/s) 198.26271089688606
have process: 143100 speed:(instance/s) 198.20071583979305
have process: 143200 speed:(instance/s) 198.20270063305773
have process: 143300 speed:(instance/s) 198.21016902700322
have process: 143400 speed:(instance/s) 198.2381995739099
have process: 143500 speed:(instance/s) 198.23343693137048
h